<a href="https://colab.research.google.com/github/gen-ai-capstone-project-bartender-agent/MOK-5-ha/blob/main/notebooks/gradio_ui_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing for Kaggle Submission

This notebook is primarily for testing Gradio UI in-notebook. Note that this is not the only valid way in which we can test our use of Gradio, but rather that once we acheieve a desired result in an IDE, we must ensure it can be implemented here as well.

In [1]:
# Remove conflicting packages from the Kaggle base environment.
!pip uninstall -qqy thinc spacy fastai google-cloud-bigquery
!pip install "google-generativeai>=0.3.0" "langgraph>=0.0.10" "requests>=2.31.0" "websockets>=12.0" "tenacity>=8.2.3" "gradio>=4.0.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 20.8 MB/s eta 0:00:00


Simply add your Gemini API key to the 'key' icon on the left sidebar and you can run the notebook.

In [2]:
# --- Imports ---
import logging
import os
import sys
from typing import Dict, List, Optional

# UI / Display
import gradio as gr
# from IPython.display import Markdown, display # Not needed for Gradio script

# Gemini - Frontier LLM
try:
    # Using 'ggenai' alias consistent with user's snippets
    import google.generativeai as ggenai
    from google.api_core import retry as core_retry # For potential core retries
    from google.generativeai import types as genai_types # For specific types if needed later
    #from google.generativeai import errors as genai_errors # For specific error handling
except ImportError:
    print("Error: google.generativeai library not found.")
    print("Please install it using: pip install google-generativeai")
    sys.exit(1)

# Tenacity for retries on specific functions
from tenacity import (
    retry as tenacity_retry, # Alias to avoid confusion with google.api_core.retry
    stop_after_attempt,
    wait_exponential,
    retry_if_exception_type,
    before_sleep_log
)

# Attempt to import userdata for Colab, fallback to environment variables
try:
    from google.colab import userdata
    IS_COLAB = True
except ImportError:
    IS_COLAB = False
    # Consider adding python-dotenv for local .env file support
    # from dotenv import load_dotenv
    # load_dotenv()

# --- Configuration ---

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Get API Key
GOOGLE_API_KEY = None
if IS_COLAB:
    try:
        GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
        logger.info("Retrieved GOOGLE_API_KEY from Colab userdata.")
    except Exception as e:
        logger.warning(f"Could not get GOOGLE_API_KEY from Colab userdata: {e}")

if not GOOGLE_API_KEY:
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
    if GOOGLE_API_KEY:
        logger.info("Retrieved GOOGLE_API_KEY from environment variable.")

if not GOOGLE_API_KEY:
    logger.error("FATAL: GOOGLE_API_KEY not found in Colab userdata or environment variables.")
    # You might want to exit or raise an error here depending on desired behavior
    raise EnvironmentError("GOOGLE_API_KEY is required but not found.")


# Configure Gemini Client and Model
try:
    ggenai.configure(api_key=GOOGLE_API_KEY)
    # Use a valid and available model name, e.g., 'gemini-1.5-flash' or 'gemini-pro'
    # 'gemini-2.0-flash' is not a standard public model name as of late 2024
    MODEL_NAME = 'gemini-2.0-flash'
    model = ggenai.GenerativeModel(MODEL_NAME)
    logger.info(f"Successfully initialized Gemini model: {MODEL_NAME}")

    # Optional: Apply retry logic directly to the client's method if desired
    # is_retriable = lambda e: isinstance(e, genai_errors.ResourceExhaustedError) or \
    #                           isinstance(e, genai_errors.InternalServerError) or \
    #                           isinstance(e, genai_errors.ServiceUnavailableError)
    # model.generate_content = core_retry.Retry(predicate=is_retriable)(model.generate_content)
    # logger.info("Applied google.api_core retry logic to generate_content.")

except Exception as e:
    logger.exception(f"Fatal: Failed to initialize Gemini model: {str(e)}")
    raise RuntimeError(
        f"Failed to initialize Gemini model. Check API key and model name ('{MODEL_NAME}')."
    ) from e


# --- Module-Level State Variables ---

# Define the Menu
menu: Dict[str, Dict[str, float]] = {
    "1": {"name": "Old Fashioned", "price": 12.00},
    "2": {"name": "Margarita", "price": 10.00},
    "3": {"name": "Mojito", "price": 11.00},
    "4": {"name": "Martini", "price": 13.00},
    "5": {"name": "Whiskey Sour", "price": 11.00},
    "6": {"name": "Gin and Tonic", "price": 9.00},
    "7": {"name": "Manhattan", "price": 12.00},
    "8": {"name": "Daiquiri", "price": 10.00},
    "9": {"name": "Negroni", "price": 11.00},
    "10": {"name": "Cosmopolitan", "price": 12.00}
}

# Define Current Order and Conversation History (mutable state)
current_order: List[Dict[str, float]] = []
# Storing history in the format Gradio expects: [{'role': 'user'/'assistant', 'content': '...'}]
conversation_history: List[Dict[str, str]] = []

# --- Core Agent Functions ---

def get_menu_text() -> str:
    """Generates the menu text."""
    global menu
    menu_text = "Menu:\n" + "-"*5 + "\n"
    for item_id, item in menu.items():
        menu_text += f"{item_id}. {item['name']} - ${item['price']:.2f}\n"
    return menu_text

# Define specific exceptions for tenacity retry relevant to API calls
# RETRYABLE_EXCEPTIONS = (
    # genai_errors.ResourceExhaustedError,
    # genai_errors.InternalServerError,
    # genai_errors.ServiceUnavailableError,
    # Add other potentially transient network errors if needed, e.g., ConnectionError
    # ConnectionError, # Be cautious with retrying generic connection errors
# )

@tenacity_retry(
    stop=stop_after_attempt(3),
    wait=wait_exponential(multiplier=1, min=2, max=10), # Adjusted wait time
    # etry=retry_if_exception_type(RETRYABLE_EXCEPTIONS),
    before_sleep=before_sleep_log(logger, logging.WARNING),
    reraise=True # Re-raise the exception if all retries fail
)
def _call_gemini_api(prompt_content: List[str], config: Dict) -> ggenai.types.GenerateContentResponse:
    """Internal function to call the Gemini API with retry logic."""
    logger.debug("Calling Gemini API...")
    response = model.generate_content(
        contents=prompt_content, # Correct parameter name is 'contents'
        generation_config=config,
        # safety_settings can be added here if needed
    )
    logger.debug("Gemini API call successful.")
    return response


def process_order(user_input_text: str) -> str:
    """
    Processes user input, calls Gemini, updates state, and returns the response text.
    Manages the module-level 'conversation_history' and 'current_order'.
    """
    global conversation_history, current_order, menu, model # Declare modification/access

    if not user_input_text:
        logger.warning("Received empty user input.")
        return "Please tell me what you'd like to order."

    try:
        # --- Construct the prompt for Gemini ---
        # Use the existing conversation history managed globally
        prompt_context = [
            "You are a friendly and helpful bartender taking drink orders.",
            "Be conversational. Ask clarifying questions if the order is unclear.",
            "If the user asks for something not on the menu, politely tell them and show the menu again.",
            "If the user asks to see their current order, list the items and their prices.",
            "\nHere is the menu:",
            get_menu_text(),
            "\nCurrent order:",
        ]
        if current_order:
            order_text = "\n".join([f"- {item['name']} (${item['price']:.2f})" for item in current_order])
            prompt_context.append(order_text)
        else:
            prompt_context.append("No items ordered yet.")

        prompt_context.append("\nConversation History (latest turns):")

        # Create prompt history from the global history (Gradio format)
        history_limit = 10 # Keep the last ~5 pairs of interactions
        limited_history_for_prompt = conversation_history[-history_limit:]

        for entry in limited_history_for_prompt:
             role = entry.get("role", "unknown").capitalize()
             content = entry.get("content", "")
             prompt_context.append(f"{role}: {content}")

        # Add the current user input to the prompt context
        prompt_context.append(f"\nUser: {user_input_text}")
        prompt_context.append("\nBartender:") # Ask the model to reply as the bartender

        full_prompt = "\n".join(prompt_context)
        logger.info(f"Processing user input: {user_input_text}")
        logger.debug(f"Full prompt for Gemini:\n------\n{full_prompt}\n------")

        # --- Call the Gemini model via the retry wrapper ---
        config_dict = {
            'temperature': 0.7,
            'max_output_tokens': 2048,
            # 'candidate_count': 1 # Usually defaults to 1
        }

        # Note: 'contents' expects an iterable of 'Content' parts.
        # For simple text prompts, passing the string directly often works,
        # but wrapping in a list is safer.
        response = _call_gemini_api(prompt_content=[full_prompt], config=config_dict)

        # --- Process the response ---
        agent_response_text = "" # Default empty response

        # Check response validity and safety
        if not response.candidates:
             logger.error("Gemini response has no candidates.")
             if response.prompt_feedback and response.prompt_feedback.block_reason:
                 logger.error(f"Prompt Blocked: {response.prompt_feedback.block_reason_message}")
                 agent_response_text = f"I'm sorry, my ability to respond was blocked. Reason: {response.prompt_feedback.block_reason_message or response.prompt_feedback.block_reason}"
             else:
                 agent_response_text = "Sorry, I couldn't generate a response. Please try again."

        elif not response.candidates[0].content or not response.candidates[0].content.parts:
             logger.error("Gemini response candidate is empty or has no parts.")
             finish_reason = response.candidates[0].finish_reason
             finish_reason_name = finish_reason.name if finish_reason else 'UNKNOWN'
             logger.error(f"Finish Reason: {finish_reason_name}")

             if finish_reason_name == "SAFETY":
                 agent_response_text = "I'm sorry, I can't provide that response due to safety reasons."
             elif finish_reason_name == "RECITATION":
                 agent_response_text = "My response couldn't be completed due to potential recitation issues."
             elif finish_reason_name == "MAX_TOKENS":
                 # Attempt to get partial text if stopped due to length
                 try:
                     agent_response_text = response.candidates[0].content.parts[0].text + "... (response truncated)"
                     logger.warning("Response truncated due to max_tokens.")
                 except (AttributeError, IndexError):
                     agent_response_text = "My response was cut short as it reached the maximum length."
             else:
                agent_response_text = f"Sorry, I had trouble generating a complete response (Finish Reason: {finish_reason_name}). Could you rephrase?"
        else:
             # Successfully got response text
             agent_response_text = response.candidates[0].content.parts[0].text
             logger.info(f"Gemini response received: {agent_response_text}")

             # --- Update Order Based on Response (Simple Heuristic) ---
             # This logic remains basic and might need refinement.
             for item_id, item in menu.items():
                 item_name_lower = item["name"].lower()
                 response_lower = agent_response_text.lower()
                 # Check if item name is mentioned and there's indication of adding
                 if item_name_lower in response_lower and \
                    any(add_word in response_lower for add_word in ["added", "adding", "got it", "sure thing", "order up", "coming right up"]):
                      # Avoid adding duplicates if it's already the *last* item added
                      if not current_order or item["name"] != current_order[-1]["name"]:
                          current_order.append(item)
                          logger.info(f"Heuristic: Added '{item['name']}' to order based on Gemini response.")
                          break # Only add the first match found

        # --- Update Global Conversation History ---
        # Add the user input *before* the assistant response for correct ordering
        # Check if the last entry was already this user's input (e.g., retry scenario)
        if not conversation_history or \
           conversation_history[-1]['role'] != 'user' or \
           conversation_history[-1]['content'] != user_input_text:
             conversation_history.append({'role': 'user', 'content': user_input_text})

        # Add the agent's response
        conversation_history.append({'role': 'assistant', 'content': agent_response_text})

        return agent_response_text

    except Exception as e:
        # Catch exceptions not handled by tenacity retry (e.g., programming errors, unexpected API issues)
        logger.exception(f"Critical error in process_order: {str(e)}")
        # Provide a safe fallback response
        return "I'm sorry, an unexpected error occurred. Please try again later."


def reset_order():
    """Resets the current order and conversation history."""
    global current_order, conversation_history
    try:
        current_order = []
        conversation_history = [] # Clear the global history
        logger.info("Order and conversation history reset successfully.")
    except Exception as e:
        logger.exception(f"Error resetting order state: {str(e)}")
        # This shouldn't really fail, but log just in case
        # Avoid raising runtime error here to keep UI responsive if possible


# --- Gradio Interface Callbacks ---

def handle_gradio_input(user_input, chat_display_history):
    """
    Gradio callback: Takes user input and current UI history,
    calls the agent, and returns updated UI state.
    """
    logger.info(f"Gradio input: '{user_input}'")
    logger.debug(f"Gradio display history received (type: {type(chat_display_history)}): {chat_display_history}")

    # Call the main processing function which uses/updates global state
    # Ensure process_order always returns a string, even on error
    response_text = process_order(user_input)
    if response_text is None: # Add safety check
        logger.error("process_order returned None! Defaulting to error message.")
        response_text = "An internal error occurred (process_order returned None)."
        # Optionally update history here too, or rely on process_order's error handling
        if not conversation_history or conversation_history[-1]['role'] != 'assistant':
             conversation_history.append({'role': 'assistant', 'content': response_text})


    # The global 'conversation_history' is now the source of truth.
    # Return the entire updated global history to Gradio.
    # Gradio's chatbot component expects a list of dicts: [{'role': 'user'/'assistant', 'content': '...'}]

    # --- Added Debugging ---
    logger.debug(f"Value of global conversation_history before return (type: {type(conversation_history)}): {conversation_history}")
    if conversation_history is None:
        logger.error("CRITICAL: Global conversation_history is None before returning to Gradio!")
        # Decide how to handle this - maybe return empty list to prevent crash?
        history_to_return = []
    else:
        history_to_return = conversation_history
    # --- End Added Debugging ---


    # Return empty string to clear input box, and the updated history list
    return "", history_to_return # Use the potentially corrected history_to_return


def clear_chat_state():
    """Gradio callback: Clears backend state and returns empty list for UI."""
    logger.info("Clear button clicked.")
    reset_order() # Clears global current_order and conversation_history
    return [] # Return empty list to clear the Gradio chatbot display


# --- Gradio UI Definition ---

with gr.Blocks(theme=gr.themes.Soft()) as demo: # Added a theme
    gr.Markdown("# Bartending Agent")
    gr.Markdown("Welcome! Ask me for a drink from the menu or check your order.")
    with gr.Row():
        with gr.Column(scale=2):
            chatbot = gr.Chatbot(
                [], # Initialize empty, will be populated by handle_gradio_input's return value
                elem_id="chatbot",
                label="Conversation",
                bubble_full_width=False,
                height=500,
                # value=conversation_history # Initial value if needed, but usually starts empty
                type="messages" # Explicitly set type
            )
            msg_input = gr.Textbox(
                label="Your Order / Message",
                placeholder="What can I get for you? (e.g., 'I'd like an Old Fashioned', 'What's on the menu?', 'Show my order')"
            )
            with gr.Row():
                clear_btn = gr.Button("Clear Conversation")
                submit_btn = gr.Button("Send", variant="primary") # Added explicit submit button

        with gr.Column(scale=1):
             gr.Markdown("### Menu")
             # Display menu dynamically using the function
             gr.Markdown(get_menu_text(), elem_id="menu-display")
             # Could add components to display current order here if desired

    # --- Event Handlers ---
    # Link Textbox submit (Enter key) to handle_gradio_input
    msg_input.submit(
        handle_gradio_input,
        [msg_input, chatbot], # Inputs: current message text, current chatbot state
        [msg_input, chatbot]  # Outputs: clear message box, updated chatbot state
    )

    # Link Submit button click to handle_gradio_input
    submit_btn.click(
        handle_gradio_input,
        [msg_input, chatbot],
        [msg_input, chatbot]
    )

    # Link Clear button click to clear_chat_state
    clear_btn.click(
        clear_chat_state,
        None,               # No inputs needed for clear
        [chatbot]           # Output: clear the chatbot display
    )


# --- Launch the Gradio Interface ---
if __name__ == "__main__":
    logger.info("Launching Gradio interface...")
    # debug=True enables auto-reloading and more logs (good for dev)
    # share=True creates a public link (use False for local-only)
    demo.launch(debug=True, share=True)
    logger.info("Gradio interface closed.")

<ipython-input-2-c66b5ac951e9>:350: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e7779cdbd4f100ab31.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://e7779cdbd4f100ab31.gradio.live
